#### Before You Start


cd .devcontainer </br>
run docker-compose up

In [26]:
! pip install psycopg2
! pip install sqlalchemy



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [27]:
import psycopg2  #import of the psycopg2 python library
import pandas as pd #import of the pandas python library
import pandas.io.sql as psql

##No transaction is started when commands are executed and no commit() or rollback() is required. 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [28]:
try:
    # Connect to the postgreSQL server with username, and password credentials
    con = psycopg2.connect(user = "postgres",
                                  password = "postgres",
                                  host = "localhost",
                                  port = "5432")
    
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    print("Connected Successfully to PostgreSQL server!!")
    
    # Obtain a DB Cursor to perform database operations
    cursor = con.cursor();
except (Exception, psycopg2.Error) as error :
     print ("Error while connecting to PostgreSQL", error)


Connected Successfully to PostgreSQL server!!


In [29]:
import pandas as pd
df = pd.read_csv('circuits.csv')


In [30]:
print(df.head())  # Displays the first 5 rows


   circuitId   circuitRef                            name      location  \
0          1  albert_park  Albert Park Grand Prix Circuit     Melbourne   
1          2       sepang    Sepang International Circuit  Kuala Lumpur   
2          3      bahrain   Bahrain International Circuit        Sakhir   
3          4    catalunya  Circuit de Barcelona-Catalunya      Montmeló   
4          5     istanbul                   Istanbul Park      Istanbul   

     country       lat        lng  alt  \
0  Australia -37.84970  144.96800   10   
1   Malaysia   2.76083  101.73800   18   
2    Bahrain  26.03250   50.51060    7   
3      Spain  41.57000    2.26111  109   
4     Turkey  40.95170   29.40500  130   

                                                 url  
0  http://en.wikipedia.org/wiki/Melbourne_Grand_P...  
1  http://en.wikipedia.org/wiki/Sepang_Internatio...  
2  http://en.wikipedia.org/wiki/Bahrain_Internati...  
3  http://en.wikipedia.org/wiki/Circuit_de_Barcel...  
4         http://en.w

### Create sample table

In [31]:
#Create "test" Table

try:
    #table_name variable
    seasonTable="season"
    create_seasonTable_query = '''CREATE TABLE '''+ seasonTable+''' 
              (year INT PRIMARY KEY,
                url TEXT
               ); '''

    #Execute this command (SQL Query)
    cursor.execute(create_seasonTable_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("Table ("+ seasonTable +") created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print("Error While Creating the DB: ",error)

Error While Creating the DB:  relation "season" already exists



Check table creation

In [32]:
# [information_schema.tables] keep listing of every table being managed by Postgres for a particular database.
# specifying the tabel_schema to 'public' to only list tables that you create.
cursor.execute("""SELECT table_name 
                  FROM information_schema.tables 
                  WHERE table_schema = 'public'  
               """)

for table in cursor.fetchall():
    print(table)

('season',)
('racestest',)


In [33]:
print("Databases in the PostgreSQL instance:")
cursor.execute("SELECT datname FROM pg_database WHERE datistemplate = false;")
databases = cursor.fetchall()
for db in databases:
    print("-", db[0])

Databases in the PostgreSQL instance:
- postgres


Test entering values

In [34]:
sql_insert_customers = "INSERT INTO season (year,url) VALUES(%s,%s)"

#list of customers (With Their attributes values)
#None here means value will be missing in the table
customer_List=[
            (200000, "mohamed.ragb@ut.ee"),
            (2, "j.smith@hotmail.com"),
            (3, None),
            (4, "jeanlime@gmail.com"),
            (5,None)]

try:
    # execute the INSERT statement
    cursor.executemany(sql_insert_customers,customer_List)
    # commit the changes to the database
    con.commit()
    #the number of inserted rows/tuples
    count = cursor.rowcount
    print (count, "Record inserted successfully into customers table")

except (Exception, psycopg2.Error) as error :
    con.rollback()
    print ("Error while Inserting the data to the table, Details: ",error)

Error while Inserting the data to the table, Details:  duplicate key value violates unique constraint "season_pkey"
DETAIL:  Key (year)=(200000) already exists.



In [35]:
sql_select_query = """ SELECT * FROM racestest """

try:
    
    cursor.execute(sql_select_query, (1,))
    person_records = cursor.fetchall() 
    print("Print each row and it's columns values:\n")
    for row in person_records:
        print("year = ", row[0], )
        print("url = ", row[1], )
except(Exception, psycopg2.Error) as error :
    con.rollback()
    print("Error:", error)

Print each row and it's columns values:

year =  1
url =  2009
year =  2
url =  2009
year =  3
url =  2009
year =  4
url =  2009
year =  5
url =  2009
year =  6
url =  2009
year =  7
url =  2009
year =  8
url =  2009
year =  9
url =  2009
year =  10
url =  2009
year =  11
url =  2009
year =  12
url =  2009
year =  13
url =  2009
year =  14
url =  2009
year =  15
url =  2009
year =  16
url =  2009
year =  17
url =  2009
year =  18
url =  2008
year =  19
url =  2008
year =  20
url =  2008
year =  21
url =  2008
year =  22
url =  2008
year =  23
url =  2008
year =  24
url =  2008
year =  25
url =  2008
year =  26
url =  2008
year =  27
url =  2008
year =  28
url =  2008
year =  29
url =  2008
year =  30
url =  2008
year =  31
url =  2008
year =  32
url =  2008
year =  33
url =  2008
year =  34
url =  2008
year =  35
url =  2008
year =  36
url =  2007
year =  37
url =  2007
year =  38
url =  2007
year =  39
url =  2007
year =  40
url =  2007
year =  41
url =  2007
year =  42
url =  2007
ye

In [36]:
import pandas as pd
from sqlalchemy import create_engine

# Step 1: Load the CSV file into a pandas DataFrame
csv_file_path = "races.csv"  # Replace with your CSV file path
table_name = "racestest"  # Replace with your PostgreSQL table name
df = pd.read_csv(csv_file_path)

# Step 2: Connect to the PostgreSQL database
# Replace the placeholders with your database credentials
user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
database = "postgres"
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Step 3: Write the DataFrame to the PostgreSQL table
try:
    df.to_sql(table_name, con=engine, if_exists="replace", index=False)
    print(f"Data from {csv_file_path} successfully imported into {table_name}!")
except Exception as e:
    print(f"An error occurred: {e}")


Data from races.csv successfully imported into racestest!
